# Data Combination

In this exercise we received 3 files about the sales of chocolate in 2020 from the imaginary company Sprint & Lüngli. The company sells 3 types of chocolate: normal, fancy and frozen. Before doing some machine learning with it (next lesson) we want to combine the files in one dataframe. However, the files are in different formats and contain different parts of the data. The exercise is the following: Take the 3 files chocolate1.csv, chocolate2.xlsx and chocolate3.xlsx and create one new dataframe that contains the combination of all information in the three files.

Hint1: chocolate1.csv has the time in the format "%d/%m/%Y" and datetime.strptime can convert it to a python datetime object.

Hint2: datetime.datetime(year, month, day) might help for chocolate2.xlsx

Hint3: We are only interested in the sales in 2020, other data can be ignored